In [10]:
import sys
sys.path.append("../../")

In [11]:
from scipy.signal import find_peaks
import librosa
import os.path
import pickle as pkl
import json

sys.path.append("./dependencies/hps")
sys.path.append("./dependencies/hps/preprocess")

from xsens import mvnx
from unity_functions import process_smpl_pose, process_smpl_trans

In [12]:
import numpy as np

In [13]:
# mvnx_name = "./SUB2_MPI_EG-001.mvnx" # read error 
mvnx_name = "./SUB5_MPI_GEB-002.mvnx"

basename = os.path.splitext( os.path.basename(mvnx_name) )[0]
pose_txt_file = f"{basename}_pose.txt"
trans_txt_file = f"{basename}_trans.txt"

mvnx_data = mvnx.MVNX(mvnx_name)

In [14]:
len(mvnx_data.frames)
mvnx_data.available_info

['acceleration',
 'angularAcceleration',
 'angularVelocity',
 'centerOfMass',
 'footContacts',
 'jointAngle',
 'jointAngleErgo',
 'jointAngleErgoXZY',
 'jointAngleXZY',
 'orientation',
 'position',
 'sensorFreeAcceleration',
 'sensorMagneticField',
 'sensorOrientation',
 'velocity']

In [15]:
mvnx_data.total_frames

3687

In [16]:
t = 0
len(mvnx_data.frames), type(mvnx_data.frames[t]), mvnx_data.frames[t]["time"], mvnx_data.frames[t]["ms"]

t = 60
len(mvnx_data.frames), type(mvnx_data.frames[t]), mvnx_data.frames[t]["time"], mvnx_data.frames[t]["ms"]

(3687, untangle.Element, '2000', '1604149861413')

In [17]:
ms_times = [ int(frame["ms"]) for frame in mvnx_data.frames ]

In [65]:
quat = mvnx_data.get_info("orientation")

In [66]:
acc = mvnx_data.get_info("acceleration")

In [69]:
quat[0], acc[0]

({'pelvis': [0.146689, -0.141488, 0.00752, 0.978983],
  'l5': [0.138053, -0.085788, -0.00034, 0.986702],
  'l3': [0.133698, -0.010584, -0.010643, 0.990908],
  't12': [0.129452, 0.01442, -0.01393, 0.991383],
  't8': [-0.126882, 0.017345, 0.009942, -0.991716],
  'neck': [-0.145733, 0.15478, -0.008888, -0.977101],
  'head': [0.180519, -0.100569, -0.002003, 0.978414],
  'right_shoulder': [0.140998, -0.06906, 0.06723, 0.985307],
  'right_upper_arm': [0.050551, 0.186772, 0.643774, 0.740348],
  'right_forearm': [-0.536922, 0.496165, 0.548569, 0.405719],
  'right_hand': [0.57484, -0.614054, -0.525264, -0.128815],
  'left_shoulder': [-0.134743, 0.059856, 0.074534, -0.986259],
  'left_upper_arm': [-0.183973, 0.018805, 0.700855, -0.688914],
  'left_forearm': [-0.468381, -0.442515, 0.739442, -0.195002],
  'left_hand': [0.439298, 0.466427, -0.763038, 0.08506],
  'right_upper_leg': [0.146245, 0.651969, -0.170531, 0.724202],
  'right_lower_leg': [-0.206971, -0.352764, 0.090998, -0.907987],
  'right_f

In [7]:
# https://github.com/jyf588/transformer-inertial-poser/blob/adef2489c927413228ed4137c8fc58e74b58fa11/data-gen-and-viz-bullet-new.py#L157

#     if USE_KNEE_RATHER_ANKLE_IMU:
#         imu_joints = [
#             _info.root,
#             _info.lwrist,
#             _info.rwrist,
#             _info.lknee,
#             _info.rknee,
#             _info.upperneck,
#         ]
#     else:
#         imu_joints = [
#             _info.root,
#             _info.rankle,
#             _info.lankle,
#             _info.lwrist,
#             _info.rwrist,
#             _info.upperneck
#         ]

In [72]:
# USE_KNEE_RATHER_ANKLE_IMU = True
USE_KNEE_RATHER_ANKLE_IMU = False

if USE_KNEE_RATHER_ANKLE_IMU:
    SPARSE_JOINT_IMUS = [ 
        "pelvis", 
        "left_forearm", 
        "right_forearm",
        "left_upper_leg", 
        "right_upper_leg",
        "head"]
else:
    SPARSE_JOINT_IMUS = [ 
        "pelvis", 
        "left_forearm", 
        "right_forearm",
        "left_lower_leg", 
        "right_lower_leg",
        "head"]
    
def get_sparse_data(data):
    sparse_data = []
    for t,t_data in enumerate(data):
        sparse_data.append( [ t_data[key] for key in SPARSE_JOINT_IMUS ] )
    return np.asarray(sparse_data)

In [73]:
sparse_quat = get_sparse_imu_data(quat)

In [74]:
sparse_acc = get_sparse_imu_data(acc)

In [76]:
np.concatenate([sparse_quat, sparse_acc], axis=2).shape 

(3687, 6, 7)

In [10]:
# necessary

# initial pose : format -> amass_char_info ?
#  get from smpl.txt in HPS

poses = process_smpl_pose(pose_txt_file)
trans = process_smpl_trans(trans_txt_file)

In [11]:
poses.shape, trans.shape

((3683, 72), (3683, 3))

In [12]:
#     from smplpytorch.pytorch.smpl_layer import SMPL_Layer

#     def bake(imu_dict, TRANS):
#         """
#         Bakes the imu dictionary by multiplying each pose and distance by the TRANS matrix
#         :param imu_dict: the input imu dictionary
#         :param TRANS: the transformation matrix
#         :return: The transformed dictionary
#         """
#         imu_poses = imu_dict['poses']
#         imu_transes = imu_dict['transes']
#         imu_transes_save = np.copy(imu_transes)
#         for i in range(imu_poses.shape[0]):
#             glob_or = imu_poses[i, :3]
#             rot_matrix = R.from_rotvec(glob_or).as_matrix()
#             new_glob = np.matmul(TRANS, rot_matrix)
#             new_glob_ax = R.from_matrix(new_glob).as_rotvec()
#             imu_poses[i, :3] = new_glob_ax

#             trans = np.reshape(imu_transes[i], (3, 1))
#             imu_transes_save[i] = np.reshape(np.matmul(TRANS, trans), (3,))

#         ret_dict = {
#                     'poses': imu_poses,
#                     'transes': imu_transes_save
#                     }

#         return ret_dict

In [13]:
# MODEL_PATH = "./smpl/models"
# # gender = "neutral" # doesn't exist
# gender = "male"

# smpl_layer = SMPL_Layer(
#             center_idx=0,
#             gender=gender,
#             model_root=MODEL_PATH)

In [14]:
import os

filename = os.path.splitext( os.path.basename(mvnx_name) )[0]
np.save(f"sparse_acc--{filename}.npy", sparse_acc)

with open("sparse_acc-def.txt", "w") as f:
    f.write( "\n".join(SPARSE_JOINT_IMUS) )

In [15]:
betas = np.zeros(16)

motion_file = f"smpl--{filename}.npz"
np.savez(motion_file, 
        poses=poses,
        trans=trans,
        betas=betas,
        mocap_framerate=60)

In [16]:
AMASS_data_filename = "../../data/source/AMASS_CMU/01/01_01_poses.npz"
amass_data = np.load(AMASS_data_filename)

In [17]:
amass_data.files

['trans', 'gender', 'mocap_framerate', 'betas', 'dmpls', 'poses']

In [18]:
amass_data["poses"].shape, amass_data["trans"].shape, amass_data["betas"].shape

((2751, 156), (2751, 3), (16,))

In [19]:
from fairmotion.data import amass

In [20]:
motion = amass.load(file=motion_file, 
               bm_path="../../data/smplh/neutral/model.npz", 
               model_type="smpl")

In [21]:
import importlib.util
from typing import List, Dict
import random

import pybullet as pb
import pybullet_data
from fairmotion.core.motion import Motion

import bullet_client
from bullet_agent import SimAgent
import constants as cst

In [22]:
# !ls ../../

In [23]:
# AMASS_URDF_FILE = "data/amass.urdf"
AMASS_URDF_FILE = "../../data/amass.urdf"

In [24]:
spec = importlib.util.spec_from_file_location(
    "char_info", "../../amass_char_info.py")
char_info = importlib.util.module_from_spec(spec)
spec.loader.exec_module(char_info)

In [25]:
        h = cst.NOMINAL_H * random.uniform(0.9, 1.1)

        Mode = pb.DIRECT
        pb_client = bullet_client.BulletClient(
            connection_mode=Mode)
        pb_client.setAdditionalSearchPath(pybullet_data.getDataPath())
        pb_client.resetSimulation()
        
        robot = SimAgent(name='sim_agent_0',
                         pybullet_client=pb_client,
                         model_file=AMASS_URDF_FILE,
                         char_info=char_info,
                         scale=h / 1.6,
                         ref_scale=h / 1.6,
                         self_collision=False,
                         # actuation=spd,
                         kinematic_only=True,
                         verbose=True)        

[SimAgent] Creating an agent... ../../data/amass.urdf


In [26]:
def get_raw_motion_info(robot: SimAgent, m: Motion) -> List[Dict[int, List[float]]]:
    # runs robot FK
    raw_info = []
    cur_time = 0.015 / 2.0

    while cur_time < m.length():
        cur_pose = m.get_pose_by_time(cur_time)
        robot.set_pose(cur_pose, None)

        cur_info = {}

        all_joint_idx = robot.get_char_info().joint_idx.values()

        for idx in all_joint_idx:
            if idx == robot.get_char_info().root:
                # this is the root joint
                _, Q, _, _ = robot.get_root_state()
                p = robot.get_root_local_point_p(cst.ROOT_COM_OFFSET)
                cur_info[idx] = list(p) + list(Q)
            else:
                state = robot.get_link_states([idx])
                cur_info[idx] = list(state[0]) + list(state[1])
            assert len(cur_info[idx]) == 7

        # print(cur_info)
        # input("press enter")
        raw_info.append(cur_info)
        cur_time += cst.DT

    return raw_info

In [27]:
motion_info = get_raw_motion_info(robot, motion)

In [28]:
# len(motion_info)
motion_info[0].keys()
motion_info[1].keys()
len(motion_info[1][-1])

7

In [29]:
# get xyz init!

In [30]:
# inference

In [42]:
from typing import Union, Tuple

import torch
from torch import nn

from real_time_runner_minimal import RTRunnerMin

In [43]:
def run_ours_wrapper_with_c_rt(imu, s_gt, model_name, char) -> (np.ndarray, np.ndarray):
    def load_model(name):
        from simple_transformer_with_state import TF_RNN_Past_State
        input_channels_imu = 6 * (9 + 3)
        if USE_5_SBP:
            output_channels = 18 * 6 + 3 + 20
        else:
            output_channels = 18 * 6 + 3 + 8

        model = TF_RNN_Past_State(
            input_channels_imu, output_channels,
            rnn_hid_size=512,
            tf_hid_size=1024, tf_in_dim=256,
            n_heads=16, tf_layers=4,
            dropout=0.0, in_dropout=0.0,
            past_state_dropout=0.8,
            with_acc_sum=WITH_ACC_SUM
        )
        model.load_state_dict(torch.load(name))
        model = model.cuda()
        # model.eval()
        return model

    m = load_model(model_name)

    # ours_out, c_out, viz_locs_out = test_run_ours_gpt_v4_with_c_rt(char, s_gt, imu, m, 40)
    ours_out, c_out, viz_locs_out = test_run_ours_gpt_v4_with_c_rt_minimal(char, s_gt, imu, m, 40)

    return ours_out, c_out, viz_locs_out


def test_run_ours_gpt_v4_with_c_rt_minimal(
        char: SimAgent,
        s_gt: np.array,
        imu: np.array,
        m: nn.Module,
        max_win_len: int
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:

    # use real time runner with offline data
    rt_runner = RTRunnerMin(
        char, m, max_win_len, s_gt[0],
        with_acc_sum=WITH_ACC_SUM,
    )

    m_len = imu.shape[0]
    s_traj_pred = np.zeros((m_len, cst.n_dofs * 2))
    s_traj_pred[0] = s_gt[0]

    c_traj_pred = np.zeros((m_len, rt_runner.n_sbps * 4))
    viz_locs_seq = [np.ones((rt_runner.n_sbps, 3)) * 100.0]

    for t in range(0, m_len-1):
        res = rt_runner.step(imu[t, :], s_traj_pred[t, :3])

        s_traj_pred[t + 1, :] = res['qdq']
        c_traj_pred[t + 1, :] = res['ct']

        viz_locs = res['viz_locs']
        for sbp_i in range(viz_locs.shape[0]):
            viz_point(viz_locs[sbp_i, :], sbp_i)
        viz_locs_seq.append(viz_locs)

        if RENDER:
            time.sleep(1. / 180)

    # throw away first "trim" predictions (our algorithm gives dummy values)... append dummy value in the end.
    viz_locs_seq = np.array(viz_locs_seq)
    assert len(viz_locs_seq) == len(s_traj_pred)

    # +2 because post-processing moving average filter effectively introduce a bit more delay
    trim = rt_runner.IMU_n_smooth + 2
    s_traj_pred[0:-trim, :] = s_traj_pred[trim:, :]
    s_traj_pred[-trim:, :] = s_traj_pred[-trim-1, :]
    viz_locs_seq[0:-trim, :, :] = viz_locs_seq[trim:, :, :]
    viz_locs_seq[-trim:, :, :] = viz_locs_seq[-trim-1, :, :]

    return s_traj_pred, c_traj_pred, viz_locs_seq

In [46]:
from fairmotion.ops import conversions

from data_utils import get_rot_center_sample_based, \
    get_raw_motion_info_nimble_q_dummy_dq


def get_imu_readings_from_raw_motion_info(
        robot: SimAgent,
        raw_info: List[Dict[int, List[float]]]
) -> np.ndarray:
    # a matrix of size l-by-(6*(9+3))

    l = len(raw_info)
    H = np.zeros((l, 6 * (9 + 3)), float)
    _info = robot.get_char_info()

    if USE_KNEE_RATHER_ANKLE_IMU:
        imu_joints = [
            _info.root,
            _info.lwrist,
            _info.rwrist,
            _info.lknee,
            _info.rknee,
            _info.upperneck,
        ]
    else:
        imu_joints = [
            _info.root,
            _info.rankle,
            _info.lankle,
            _info.lwrist,
            _info.rwrist,
            _info.upperneck
        ]

    for t in range(l):
        # fill in the ori readings
        imu_Rs = []  # 6*9
        cur_info = raw_info[t]

        root_q = np.array(cur_info[_info.root][3:])
        root_R = conversions.Q2R(root_q)
        imu_Rs += root_R.flatten().tolist()  # in global frame (no transform)

        for j in imu_joints[1:]:
            # other five IMUs
            joint_R = conversions.Q2R(np.array(cur_info[j][3:]))
            imu_Rs += joint_R.flatten().tolist()

        H[t, :6 * 9] = imu_Rs

    for t in range(cst.acc_fd_N, l - cst.acc_fd_N):
        # fill in the acc readings
        imu_as = []
        cur_info = raw_info[t]
        prev_info = raw_info[t - cst.acc_fd_N]
        next_info = raw_info[t + cst.acc_fd_N]

        root_a = \
            -2 * np.array(cur_info[_info.root][:3]) \
            + next_info[_info.root][:3] \
            + prev_info[_info.root][:3]
        root_a /= (cst.DT_FIN_ACC ** 2)
        imu_as += root_a.tolist()

        for j in imu_joints[1:]:
            # other five IMUs
            joint_a = -2 * np.array(cur_info[j][:3]) + next_info[j][:3] + prev_info[j][:3]
            joint_a /= (cst.DT_FIN_ACC ** 2)
            imu_as += joint_a.flatten().tolist()

        H[t, 6 * 9:] = imu_as

    # pad boundary acc
    H[:cst.acc_fd_N, 6 * 9:] = H[cst.acc_fd_N, 6 * 9:]
    H[-cst.acc_fd_N:, 6 * 9:] = H[-cst.acc_fd_N - 1, 6 * 9:]

    return H


def get_all_contr_seqs_from_raw_motion_info(
    robot: SimAgent,
    raw_info: List[Dict[int, List[float]]]
) -> np.ndarray:

    _info = robot.get_char_info()
    constr_links = [
            _info.lankle,
            _info.rankle,
            _info.lwrist,
            _info.rwrist,
            _info.root,
    ]

    c_all = []
    for link in constr_links:
        contrs_link, r_sum_link = get_link_contr_seq_from_raw_motion_info(
            raw_info, link
        )
        c_all.append(contrs_link)
        # if np.max(np.abs(r_sum_link)) > 0.2:
        #     print(link, r_sum_link)

    c_all = np.concatenate(c_all, axis=1)

    return c_all


def get_link_contr_seq_from_raw_motion_info(
        raw_info: List[Dict[int, List[float]]],
        link: int
) -> (np.ndarray, np.ndarray):
    # a l-by-4 matrix, each row being
    # (0/1, (0,0,0)/Rr)

    r_prev = None
    m_len = len(raw_info)
    contr_seq = np.zeros((m_len, 4))

    residue_sum = np.array([0., 0, 0])

    def get_p_q_from_t(_t):
        cur_info = raw_info[_t]
        link_p = np.array(cur_info[link][:3])
        link_q = np.array(cur_info[link][3:])
        return link_p, link_q

    for t in range(2, m_len - 2):

        prev_p, prev_q = get_p_q_from_t(t - 1)
        cur_p, cur_q = get_p_q_from_t(t)
        next_p, next_q = get_p_q_from_t(t + 1)

        r, residue = get_rot_center_sample_based(prev_p, prev_q, next_p, next_q, 2 * cst.DT, r_prev, link)

        residue_sum += residue * cst.DT

        # if r is None, zeros filled in already
        if r is not None:
            cur_R = conversions.Q2R(cur_q)
            assert np.linalg.norm(r) < 0.25
            contr_seq[t, :] = np.concatenate((
                [1.0],
                r
            ))

        r_prev = r

    return contr_seq, residue_sum

In [36]:
        imu = get_imu_readings_from_raw_motion_info(robot, motion_info)

        contrs = get_all_contr_seqs_from_raw_motion_info(robot, motion_info)

        qdq = get_raw_motion_info_nimble_q_dummy_dq(robot, motion)

In [48]:
import argparse


parser = argparse.ArgumentParser(description='Run our model and related works models')
parser.add_argument('--ours_path_name_kin', type=str, default="model-kin-amass-4-knee-v2.pt",
                    help='')
parser.add_argument('--name_contains', type=str, default='',
                    help='Please use "" to be able to pass multiple search keys split by whitespaces')
parser.add_argument('--test_len', type=int, default=600,
                    help='')
parser.add_argument('--render', action='store_true',
                    help='')
parser.add_argument('--compare_gt', action='store_true',
                    help='')
parser.add_argument('--seed', type=int, default=42,
                    help='')
parser.add_argument('--five_sbp', action='store_true',
                    help='')
parser.add_argument('--with_acc_sum', action='store_true',
                    help='')
parser.add_argument('--viz_terrain', action='store_true',
                    help='')
parser.add_argument('--data_version_tag', type=str, default=None,
                    help='')
# parser.add_argument('--save_c', action='store_true',
#                     help='')                # for the DIP-IMU set which has C info
# args = parser.parse_args()

import re
args = parser.parse_args(
    re.split(" ", "--ours_path_name_kin ../../output/model-new-v1-0.pt --test_len 30000 --with_acc_sum --compare_gt --seed 42 --five_sbp")
)

In [38]:
from render_funcs import init_viz, COLOR_OURS, update_height_field_pb, set_color, COLOR_GT

In [50]:
def viz_point(x, ind):
    pb_client.resetBasePositionAndOrientation(
        VIDs[ind],
        x,
        [0., 0, 0, 1]
    )

In [54]:
TEST_LEN = args.test_len
RENDER = args.render
MAX_TEST_MOTION_PRE_CAT = 50        # make testing faster
# if args.save_c:
#     MAX_TEST_MOTION_PRE_CAT = 50000
# else:
#     MAX_TEST_MOTION_PRE_CAT = 50
USE_5_SBP = args.five_sbp
WITH_ACC_SUM = args.with_acc_sum

MAP_BOUND = cst.MAP_BOUND * 2.0     # some motions are in large range
GRID_NUM = int(MAP_BOUND/cst.GRID_SIZE) * 2


# TODO: really odd, need to be huge for pybullet to work (say. 10.0)
init_grid_np = np.random.uniform(-10.0, 10.0, (GRID_NUM, GRID_NUM))
init_grid_list = list(init_grid_np.flatten())

color = COLOR_OURS

pb_client, c1, c2, VIDs, h_id, h_b_id = init_viz(char_info,
                                                 init_grid_list,
                                                 hmap_scale=cst.GRID_SIZE,
                                                 gui=RENDER,
                                                 compare_gt=args.compare_gt,
                                                 color=color,
                                                 viz_h_map=args.viz_terrain)

[SimAgent] Creating an agent... ../../data/amass.urdf
[SimAgent] Creating an agent... ../../data/amass.urdf


In [ ]:
gt_list = []
ours_list = []
ours_c_list = []
ours_c_viz_list = []
tp_list = []
dip_list = []
test_files_included = []

#     data = pickle.load(open(f, "rb"))

def inference(data):
    X = data['imu']
    Y = data['nimble_qdq']

    # exclude too short trajs
    if Y.shape[0] < 2.5 / cst.DT:
        print("cannot inference")
        return None

    # to make all motion equal in stat compute, and run faster
    if Y.shape[0] > TEST_LEN:
        rand_start = random.randrange(0, Y.shape[0] - TEST_LEN)
        start = rand_start
        end = rand_start + TEST_LEN
    else:
        start = 0
        end = Y.shape[0]
    X = X[start: end, :]
    Y = Y[start: end, :]

    # for clearer visualization, amass data not calibrated well wrt floor
    # translation errors are computed from displacement not absolute Y
    Y[:, 2] += 0.05       # move motion root 5 cm up

#     gt_list.append(Y)
#     test_files_included.append(f)
#     print(f)

    ours, C, ours_c_viz = run_ours_wrapper_with_c_rt(X, Y, args.ours_path_name_kin, c1)
#     ours_list.append(ours)
#     ours_c_viz_list.append(ours_c_viz)

    # if args.save_c:
    #     save_name = f.replace("v0", "v0_c")
    #     assert "dipimu" in f
    #     with open(save_name, "wb") as handle:
    #         assert len(X) == len(C)
    #         print(C.shape)
    #         pickle.dump(
    #             {"constrs": C},
    #             handle,
    #             protocol=pickle.HIGHEST_PROTOCOL
    #         )
    #     print("saved", save_name)
    
    return Y, ours, C, ours_c_viz

In [55]:
data = {
    "imu": imu,
    "constrs": contrs,
    "nimble_qdq": qdq,
}

Y, ours, C, ours_c_viz = inference(data)

model with acc sum
number of parameters: %e 3677315


In [58]:
Y.shape, C.shape, ours_c_viz.shape

((3682, 114), (3682, 20), (3682, 5, 3))